In [0]:
#!pip install msal
#import msal
import pandas as pd
import requests

     |████████████████████████████████| 90 kB 170 kB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-1c227a2e-88db-45de-927f-caa22b19292b/bin/python -m pip install --upgrade pip' command.


# Fucntion to connect to the API

In [0]:
#Endpoint API
url = "https://us-central1-bix-tecnologia-prd.cloudfunctions.net/api_challenge_junior?id="


In [0]:
def return_df(r):
    # Check if the request was successful (status code 200)
    if r.status_code == 200:
        # Convert the response JSON data to an RDD
        json_data = spark.sparkContext.parallelize([r.text])
        # Read the JSON data into a DataFrame
        df = spark.read.json(json_data)
        # Process and analyze the data as needed
        return df
    else:
        print("Error:", r.status_code)

# Loop to extract all the employees names

In [0]:
from pyspark.sql.functions import col,trim, lit
#Insert every record in a table
table = None
for line in range(1,10):
    upd_url = url+str(line)
    r = requests.get(upd_url)
    new_df = return_df(r)
    if table is None:
        table = new_df.withColumn("id", lit(line))  
    else:
        new_df_with_id = new_df.withColumn("id", lit(line))  # Add the "id" column to the new DataFrame
        table = table.union(new_df_with_id)


    

In [0]:
table.show()

+----------------+---+
| _corrupt_record| id|
+----------------+---+
|     Rob Carsson|  1|
|     Eli Preston|  2|
|    Tom Lindwall|  3|
|      Leif Shine|  4|
|  Ingrid Hendrix|  5|
|Lennart Skoglund|  6|
|    Rock Rollman|  7|
|    Helen Brolin|  8|
|    Joan Callins|  9|
+----------------+---+



#Rename columns

In [0]:

cleaned_table = table.withColumnRenamed("_corrupt_record","Employee").\
    withColumn("Employee", trim(col("Employee")))


In [0]:
cleaned_table.display()

Employee,id
Rob Carsson,1
Eli Preston,2
Tom Lindwall,3
Leif Shine,4
Ingrid Hendrix,5
Lennart Skoglund,6
Rock Rollman,7
Helen Brolin,8
Joan Callins,9


# Database connection

In [0]:
jdbcUsername = "admin-srv"
jdbcPassword = "0v954UDJ*Y2G"
jdbcHostname = "srv-fab-projetos.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "db-fab-projetos"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

table_name = 'sollim.employee'

# Insert into database

In [0]:
cleaned_table.write \
  .format("jdbc") \
  .option("url", jdbcUrl) \
  .option("dbtable", table_name) \
  .mode("overwrite") \
  .options(**connectionProperties) \
  .save()